## This notebook is a tutorial and overview of a binary mixture protocol for the OT2. Given a specified array of mole fractions and concentration of corresponding stock solutions, it will calculate the required volumes and pass that along to the OT2. While in this example it is intended for binary mixtures, it could be modified for ternary, quarternary mixtures, and the labware and pipettes should be easily modualr as well.

### First we will look at each function seperately before we introduce them into the main wrapper function.

In [1]:
#Imports
import numpy as np
from scipy.optimize import fsolve

### Pipette switching function

#### Replace the P50 and P300 with other pipettes you are using if necessary.

In [2]:
def choose_pipette(volume):
    """
    This function decides which pipette to use based on the volume given.
    Be sure the correct pipette is specified in the labware portion of the OT2 protocol for function to work.
    
    """
    values = volume
    if values == float(0):                                  # If volume is 0, well is skipped
        pass
    elif values < float(30):                                # If volume is below 30uL, P50 pipette is used
        instrument = P50
    else:                                                   # If volume is greater than 30uL, P300 pipette is used
        instrument = P300
    return instrument #returns which pipette to use

### Destination wellplate function

#### Here we are specifically using 96 well plates, replace with other wellplates if necessary. The input necessary is a list of wellplates  being used in the protocol. 

In [3]:
def destination_well_plate(i, well_plate_list):
    """
    This function checks if you've filled up the current well plate and need
    to move to on to the next one. Replace well number with any other wellplate if necessary.
    i is the count of all the wells the OT2 has filled so far (starting from 0).
    Error will be returned if you have run out of wellplates.
    
    """
    try:
        well_number = i // 96                     #floor division operator keeps track if you've filled all wells in current plate
        well_plate = well_plate_list[well_number] #value of well number will designate which plate to use based on given well plate list
    except ValueError:
          print("You ran out of space in the well plate. You either need to add new\
          well plates, or continue new experiment from where you stopped.")
    return well_plate #returns which wellplate to use

### Volume calculation function

#### Calculates the necessary volume for the OT2 to pipette based on the given mole fractions. For this to work, the array of mole fractions should look like the array below, with the mole fractions for both components in each element of the array.

In [4]:
example_array = np.array([[0.025,0.975],[0.05,0.95],[0.1, 0.9],[0.15,0.85],[0.17,0.83],[0.2,0.8],[0.25,0.75],[0.27,0.73],
                                 [0.3,0.7],[0.33,0.67],[0.35,0.65],[0.4,0.6],[0.45,0.55],[0.5,0.5],[0.55,0.45],[0.6,0.4],
                                 [0.65,0.35],[0.7,0.3],[0.75,0.25],[0.8,0.2],[0.85,0.15],[0.9,0.1],[0.95,0.05],[0.975,0.025]])

In [5]:
def calculate_volumes(DES_mole_fractions, stock_QAS, stock_HBD, total_volume):
    """
    This function calculates the necessary volume for the OT2 to pipette based on 
    a given array of mole fractions. Also necessary to give the corresponding stock
    concentrations (mol/L) of the components. Lastly, input the desired total volume of the 
    desired mixture.
    
    """                                             #empty list created to append calculated volumes in uL
    QAS = []                                        #component 1 for this case is the quarternary ammonium salt in the first index of the array element (x[0])
    HBD = []                                        #component 2 for this case is the hydrogen bond doner in the second index in the array element (x[1])
    for row in DES_mole_fractions: 
        def f(x) :                                  #system of equations to solve for necessary volume
            y = np.zeros(np.size(x))                #input desired total volume of mixture                                                                                               
            y[0] = x[0] + x[1] - total_volume       #QAS and HBD volumes will be equal to total specified                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]  #equation for mole fraction of component 1
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]  # equation for mole fraction of component 2
            return y
        x0 = np.array([100.0, 100.0, 100.0])        #input initial guesses                                   
        x = fsolve(f, x0)                           #fsolve function
        QAS.append(x[0])                            #appending QAS volumes into list
        HBD.append(x[1])                            #appending HBD volumes into list
        volumes = [QAS,HBD]                         # Appends volumes into a seperate list of lists.

    return(volumes)

#### The function will return a list of lists of volumes as below. All volumes in the first list will correspond to the QAS and the volumes in the second list will corresopond to the HBD. This allows the robot to pipette all the necessary volumes from one stock to all the mixtures, reducing the need to switch pipettes constantly. 

In [6]:
calculate_volumes(example_array, 2, 3, 300)

[[11.111111111111095,
  21.95121951219512,
  42.857142857142854,
  62.79069767002344,
  70.50691244239651,
  81.81818181818181,
  100.00000000000007,
  107.04845814977979,
  117.39130434787334,
  127.46781115879797,
  134.04255319149058,
  150.00000000019182,
  165.30612244897958,
  180.00000000001535,
  194.11764705882354,
  207.69230769230774,
  220.75471698113154,
  233.33333333333331,
  245.45454545454544,
  257.1428571428571,
  268.4210526315823,
  279.3103448284455,
  289.8305084638737,
  294.95798319328145],
 [288.88888888888886,
  278.0487804878049,
  257.14285714285717,
  237.20930232997657,
  229.4930875576035,
  218.18181818181816,
  199.99999999999994,
  192.95154185022022,
  182.60869565212664,
  172.53218884120201,
  165.95744680850942,
  149.99999999980818,
  134.6938775510204,
  119.99999999998467,
  105.88235294117648,
  92.30769230769229,
  79.24528301886848,
  66.66666666666667,
  54.54545454545455,
  42.85714285714286,
  31.578947368417715,
  20.68965517155447,
  10

### Volume transfer function

#### This function carries out the volume transfers to the appropriate wells for the entire volume list. More modifications can be made to how the transfers are carried out by referencing the OT2 API.

In [7]:
def transfer_list_of_volumes(source, starting_well_plate, starting_well_number, volume_list):
    
    """
    This function carries out the volume transfers to the appropriate wells. 
    Necessary to input a source, which will be the stock solutions. Also a 
    starting well plate from the list of well plates, and which well number
    to start from. Finally, the volume list that was calculated is needed.
    This function works together with the others to keep track of what well
    plates and wells have aklready been filled previously.
    
    """
    P300.pick_up_tip()                                   # Picks up pipette tip for both P50 and P300 to allow to alternate
    P50.pick_up_tip()
    for well_counter, values in enumerate(volume_list):
        pipette = choose_pipette(values)                 # choose pipette based on volume
        pipette.transfer(values, stock[source], starting_well_plate(starting_well_number+well_counter).top(0.5), new_tip='never') #will be pipetting from a "stock" labware specified later in OT2 protocol        
        pipette.blow_out(starting_well_plate(starting_well_number+well_counter).top(0.5)) #blows out after dispensing
    P300.drop_tip() #dropping tips when switching to a new stock solution.
    P50.drop_tip()
    return len(volume_list) #returns the length of volume list to keep track of how many wells have just been filled

### Main wrapper function

#### This will be the wrapper function to implement into an OT2 protocol. Extra print statements are included that you can use to check if your protocol is behaving appropriately and the correct wellplates, wells, stocks, etc. are being used. 

In [8]:
def main(reagent_pos, mole_fractions, well_plate_list, starting_position, stock_concentrations, total_volume):
    
    """
    This is the main wrapper function for binary mixture generation. Inputs necessary are reagent positions (essentially
    what mixtures you are making based on where they are located in the stock plate), the mole fractions desired for the
    mixtures, a list of well plates being used that must be defined beforehand in the protocol, starting position based on
    total number of available wells (i.e. start from 0 usually), a dictionary of stock concentrations, and finally the 
    total volume desired of the mixtures.

    """  
    i = starting_position                                               #based on total number of wells. For example, 384 wells in 4*96 well plates. Starting from 0 will be first plate, starting from 96 will be second plate.
    total_number_of_wells_needed = len(reagent_pos)*len(mole_fractions) #total wells will be length of mixtures array multipled by length of mole fraction array
    print('total number of wells needed is {}'.format(total_number_of_wells_needed))
    
    available_wells = (len(well_plate_list))*96 - i - 1           # checks how many wells available based on specified labware
    print('Total available wells are {}'.format(available_wells)) # This will also keep track of how many wells are left after each iteration
    
    if total_number_of_wells_needed > available_wells:            # will advise you if you need more wellplates
        print("Total number of empty wells needed for carrying out the experiment is {},\
        greater than the available empty wells {}.".format(total_number_of_wells_needed, available_wells))
        print("Either add empty wells or decrease number of mixtures")
    else:
        for j in range(len(reagent_pos)):                         #looping through all desired mixtures to be made
            Q = reagent_pos[j][0]                                 # in array of mixtures, first stock location is the QAS
            print('First reagent is {}'.format(Q))
            H = reagent_pos[j][1]                                 # The HBD is the second stock location in array of mixtures
            print('Second reagent is {}'.format(H))
            starting_well_plate = destination_well_plate(i, well_plate_list) #function determines which plate to use
            starting_well_number = i % 96                                    #modulus operator keeps track of which well to start from                               
            print('We are starting from well plate {}, and well number {}'.format(i // 96, starting_well_number))
            reagent_volume = calculate_volumes(mole_fractions, stock_concentrations[Q], stock_concentrations[H], total_volume) #calculating volumes to pipette
            print('Concentration of stock {}, is {}'.format(Q, stock_concentrations[Q]))
            print('Concentration of stock {}, is {}'.format(H, stock_concentrations[H]))                                       #reports which stocks are used, their concentrations, and the resulting calculated volume list
            print('Based on given mole fraction and stock concentration, to create total volume of {}, calculated volume list for both agents are {}'.format(total_volume, reagent_volume))
            transfer_list_of_volumes(Q, starting_well_plate,starting_well_number, reagent_volume[0])          #pipetting the volumes for the QAS (1st stock position)
            moves = transfer_list_of_volumes(H, starting_well_plate,starting_well_number, reagent_volume[1])  #pipetting the volumes for the HBD (2nd stock position)
            print('We have filled up {} wells'.format(moves))                                                 #reports how many wells we just filled in this run
            i += moves                                                                                        #This will be the next starting position for the following iteration
            print('Now we are starting from well plate {}, well number {}'.format(i // 96,i % 96))            #reports next starting well and wellplate
    return

### OT2 Protocol

#### Here is an example protocol to implement the code. Be sure to remove the robot.reset() and robot commands print statements when actually running on the robot. However, keep these when simulating protocols in jupyter notebook.

In [18]:
#Import Dependencies
from opentrons import labware, instruments, robot
import numpy as np
from scipy.optimize import fsolve
robot.reset()                                                  #remove when uploading protocol to robot
################################################################################
#Importing labware
tiprack_300 = labware.load("opentrons-tiprack-300ul", '10')    #300ul tips can be used for P300 and P50 pipettes
tiprack_300_2 = labware.load("opentrons-tiprack-300ul", '11')  #Second tiprack
stock = labware.load("trough-12row", '2' )                     #12 well resovoir for stocks listed A1-A12
A_96_well= labware.load("96-flat", '8')                        #Using 4, 96 wellplates labeled A-D
B_96_well= labware.load("96-flat", '9')
C_96_well= labware.load("96-flat", '5')
D_96_well= labware.load("96-flat", '6')
trash = robot.fixed_trash                                      #set fixed trash
################################################################################
#Importing pipettes
P300 = instruments.P300_Single(
    mount='left',
    tip_racks=[tiprack_300],
    trash_container=trash
) # Volume range = 30-300uL

P50 = instruments.P50_Single(
    mount='right',
    tip_racks=[tiprack_300_2],
    trash_container=trash
) # Volume range = 5-50uL

################################################################################
#Input volumes to pipette in uL. first list is for QAS and second is for HBD
mole_fractions = np.array([[0.025,0.975],[0.05,0.95],[0.1, 0.9],[0.15,0.85],[0.17,0.83],[0.2,0.8],[0.25,0.75],[0.27,0.73],
                                 [0.3,0.7],[0.33,0.67],[0.35,0.65],[0.4,0.6],[0.45,0.55],[0.5,0.5],[0.55,0.45],[0.6,0.4],
                                 [0.65,0.35],[0.7,0.3],[0.75,0.25],[0.8,0.2],[0.85,0.15],[0.9,0.1],[0.95,0.05],[0.975,0.025]])

################################################################################

#Define mixtures to create. A1-A12 are positions on 12 row trough where the stocks are located.
mixtures = np.array([['A1', 'A5'], ['A1', 'A6']])

################################################################################
#Create a dictionary of stock concentrations
stock_concentrations = {'A1':1, 'A2':2, 'A3':3, 'A4': 4, 'A5':1, 'A6':2}

################################################################################
#Create a wellplate list based on the previously defined labware for the code to iterate through
well_plate_list = [A_96_well, B_96_well, C_96_well, D_96_well]
#So in the list, 0 will be wellplate A, 1 will be wellplate B, and etc. 
################################################################################
robot.home() #useful to prevent collisions based on previous protocols

#Implenet the wrapper function
main(mixtures, mole_fractions, well_plate_list, 0, stock_concentrations, 300)

# for c in robot.commands():
#     print(c)


/Users/jaime/.opentrons/deck_calibration.json not found. Loading defaults
/Users/jaime/.opentrons/robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!
total number of wells needed is 48
Total available wells are 383
First reagent is A1
Second reagent is A5
We are starting from well plate 0, and well number 0
Concentration of stock A1, is 1
Concentration of stock A5, is 1
Based on given mole fraction and stock concentration, to create total volume of 300, calculated volume list for both agents are [[7.5, 15.0, 30.000000000000004, 45.00000000000001, 51.00000000000001, 60.0, 75.0, 81.0, 90.0, 99.0, 105.0, 119.99999999999856, 135.0, 150.0, 165.0, 180.0, 195.0, 210.0, 225.0, 240.00000000000003, 255.00000005698485, 270.0, 285.0, 292.5], [292.5, 285.0, 270.0, 254.99999999999997, 248.99999999999997, 240.0, 225.00000000000003, 219.0, 210.0, 201.0, 195.0, 180.00000000000142, 165.0, 150.0, 135.0, 120.0, 105.0, 90.0, 75.0, 60.0, 44.99999994301515, 30.0, 15.000000000000002, 7.500000000000001]

### In the protocol, I had defined two mixtures to make across 24 mole fractions. The print statements allow you to check if everything is working the way you want it to. We can see that we did indeed use the correct stocks, their concentrations, correct calculated volume, and how many wells we filled and where we are starting next. This should hopefully be pretty usedful. Note however, that these print statements will not appear if protocol is uploaded as a .py file in the GUI, the robot will only display explicit pipetting commands. So you can only use this to check your protocol beforehand in a jupyter notebook. To explicitly see the rest of the pipette transfer commands in the jupyter notebook as well, just uncomment  the robot.commands() for loop. 